<a href="https://colab.research.google.com/github/bruna-calistrate/DataWrangling-Case1/blob/main/Case_1_Data_Wrangling_Daily_Happiness_%26_Employee_Turnover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling e Data Cleaning

Se você fosse gerente em uma empresa, o que você faria
de diferente caso descobrisse que alguns de seus
funcionários estão pensando em sair? 

Daria um aumento, buscaria contratar outro ou tentaria ver como melhorar? De toda forma, ter essa informação seria útil e evitaria muitos problemas!

Neste exercício simularemos a etapa de Data Preparation através da limpeza e estruturação dos dados que servirão de insumo para a construção do modelo preditivo mencionado acima.



Daily Happiness & Employee Turnover


Is There a Relationship Between Employee Happiness and Job Turnover?


Dataset: https://www.kaggle.com/harriken/employeeturnover


## Importar bibliotecas

In [36]:
import pandas as pd
import numpy as np
import datetime as dt
import numpy.ma as ma
import plotly.express as px
import seaborn as sns 

## Data Understanding

### Contextualização

No verão de 2016, em Barcelona, o CEO da myhappyforce, Alex Rios, explica o funcionamento de seu aplicativo onde funcionários registram diariamente o seu nível de felicidade no trabalho. Esse aplicativo é utilizado por empresas para rastrear a felicidade de seus empregados. 

Os dados foram disponibilidados de forma anonimizada para melhor entendimento do turnover de funcionários e para possibilitar a criação de um modelo que consiga predizer quais empregados irão sair da empresa.

### Importar arquivos

**df_votes** - voto de um funcionário ao abrir o aplicativo e responder à pergunta "Quão feliz você está no trabalho hoje?". Para votar, o funcionário indica o seu sentimento tocando em um dos quatro ícones na tela.

4 colunas e 221.232 linhas
* **employee**: número do funcionário
  * tipo int, sem valores nulos
* **companyAlias**: código hash da empresa
  * tipo object, sem valores nulos
* **voteDate**: data da votação 
  * tipo object deve ser convertido pra datetime, sem valores nulos
* **vote**: voto do empregado (1-4)
  * tipo int, sem valores nulos



In [37]:
df_votes = pd.read_csv('https://raw.githubusercontent.com/bruna-calistrate/DataWrangling-Case1/main/votes.csv')

df_votes.head()

,employee,companyAlias,voteDate,vote
0,31,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
1,33,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
2,79,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
3,94,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
4,16,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,2


In [38]:
df_votes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221232 entries, 0 to 221231
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   employee      221232 non-null  int64 
 1   companyAlias  221232 non-null  object
 2   voteDate      221232 non-null  object
 3   vote          221232 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 6.8+ MB


**df_comments** - após votar, uma nova tela aparece onde o funcionário pode incluir um comentário explicando o seu sentimento. Aqui os comentários estão anonimizados.

7 colunas e 82.756 linhas
* **employee**: número do funcionário
  * tipo int, sem valores nulos
* **companyAlias**: código hash da empresa
  * tipo object, sem valores nulos
* **commentId**: código hash do comentário
 * tipo object, sem valores nulos
* **txt**: conteúdo anonimizado do comentário, com a mesma quantidade de caracteres original
 * tipo object, com alguns valores nulos
* **likes**: quantidade de curtidas
  * tipo float deve ser convertido para int, contém 1 valor nulo
* **dislikes**: quantidade de descurtidas
  * tipo float, deve ser convertido para int, contém 1 valor nulo
* **commentDate**: data de postagem do comentário
  * tipo object deve ser convertido para datetime, contém 1 valor nulo

In [39]:
df_comments = pd.read_csv('https://raw.githubusercontent.com/bruna-calistrate/DataWrangling-Case1/main/comments_clean_anonimized.csv')

df_comments.head()

,employee,companyAlias,commentId,txt,likes,dislikes,commentDate
0,307,56aec740f1ef260003e307d6,58d018d7e010990004e38070,**********************************************...,4.0,0.0,Mon Mar 20 19:00:17 CET 2017
1,382,56aec740f1ef260003e307d6,58d0179ae010990004e3806d,*****************************,1.0,2.0,Mon Mar 20 18:55:16 CET 2017
2,172,56aec740f1ef260003e307d6,58cff8cde010990004e37f6a,***************************,3.0,0.0,Mon Mar 20 16:44:02 CET 2017
3,135,56aec740f1ef260003e307d6,58cfefeee010990004e37f60,***************************,1.0,1.0,Mon Mar 20 16:06:08 CET 2017
4,225,56aec740f1ef260003e307d6,58cfd9b4e010990004e37f52,*********************************,3.0,2.0,Mon Mar 20 14:30:50 CET 2017


In [40]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82756 entries, 0 to 82755
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   employee      82756 non-null  int64  
 1   companyAlias  82756 non-null  object 
 2   commentId     82756 non-null  object 
 3   txt           82539 non-null  object 
 4   likes         82755 non-null  float64
 5   dislikes      82755 non-null  float64
 6   commentDate   82755 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 4.4+ MB


**df_interactions** - depois de comentar, uma nova tela aparece onde o funcionário pode ver os comentários de seus colegas com as ações de curtir ou descurtir.

5 colunas e 336.960 linhas
* **employee**: número do funcionário
 * tipo int, sem valores nulos
* **companyAlias**: código hash da empresa
 * tipo object, sem valores nulos
* **liked**: com ou sem curtida
 * tipo object deve ser convertido em bool, com 1 valor nulo
* **disliked**: com ou sem descurtida
 * tipo object deve ser convertido em bool, com 1 valor nulo
* **commentId**: código do comentário
 * tipo object, com 1 valor nulo

In [41]:
df_interactions = pd.read_csv('https://raw.githubusercontent.com/bruna-calistrate/DataWrangling-Case1/main/commentInteractions.csv')

df_interactions.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning:

Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.



,employee,companyAlias,liked,disliked,commentId
0,307,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
1,36,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
2,276,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
3,24,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
4,382,56aec740f1ef260003e307d6,True,False,58d0179ae010990004e3806d


In [42]:
df_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336960 entries, 0 to 336959
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   employee      336960 non-null  int64 
 1   companyAlias  336960 non-null  object
 2   liked         336959 non-null  object
 3   disliked      336959 non-null  object
 4   commentId     336959 non-null  object
dtypes: int64(1), object(4)
memory usage: 12.9+ MB


**df_churn** - status do funcionário, se continua ou não na empresa (saiu ou foi demitido)

5 colunas e 4.847 linhas
* **employee**: número do funcionário
  * tipo int, sem valores nulos
* **companyAlias**: código hash da empresa
 * tipo object, sem valores nulos
* **numVotes**: quantidade de votos
  * tipo int, sem valores nulos
* **lastPartcipationDate**: data da última participação
 * tipo object deve ser convertido para datetime, com 1 valor nulo
* **stillExists**: se o funcionário ainda trabalha na empresa ou não
 * tipo bool, sem valores nulos

In [43]:
df_churn = pd.read_csv('https://raw.githubusercontent.com/bruna-calistrate/DataWrangling-Case1/main/churn.csv')

df_churn.head()

,employee,companyAlias,numVotes,lastParticipationDate,stillExists
0,512,56aec740f1ef260003e307d6,4,Thu Feb 23 12:48:04 CET 2017,True
1,-2,56aec740f1ef260003e307d6,0,Wed Jan 18 14:00:55 CET 2017,False
2,2,56aec740f1ef260003e307d6,72,Fri Mar 17 01:00:00 CET 2017,True
3,487,56aec740f1ef260003e307d6,14,Sat Nov 19 15:02:14 CET 2016,False
4,3,56aec740f1ef260003e307d6,22,Thu Feb 16 01:00:00 CET 2017,True


In [44]:
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   employee               4847 non-null   int64 
 1   companyAlias           4847 non-null   object
 2   numVotes               4847 non-null   int64 
 3   lastParticipationDate  4847 non-null   object
 4   stillExists            4847 non-null   bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 156.3+ KB


a) Qual a sua variável target?

A feature **employee** em conjunto com a **companyAlias**, ou seja, número do funcionário por empresa, criar uma nova coluna com o agrupamento dessas informações

b) Quais as colunas do seu dataset e podem te ajudar a prever a informação desejada?

* **vote**: para avaliar a sensação de felicidade diária dos funcionários
* **commentId**: código de cada comentário
* **likes** e **dislikes**: para avaliar o quanto outros colegas concordam com a avaliação individual
* **liked** e **disliked**: funcionários que curtiram ou descurtiram comentários de seus colegas
* **numVotes**: quantidade de avaliações por funcionário

c) Existem outras informações que poderiam te ajudar a prever?

Informações sobre o mercado de trabalho na Espanha em 2016

d) Como os 4 datasets se relacionam? Quais as chaves de cada um?

Todos datasets se relacionam através da combinação das features **employee** e **companyAlias**. Em complemento, os datasets de **comments** e **interations** também se relacionam através da feature **commentId**.

## Data Preparation

* Descrição do dataset
* Seleção dos dados
* Limpeza dos dados
* Construção dos dados
* Integração dos dados
* Formatação dos dados


### Limpeza dos dados

#### Correção das informações de data

In [45]:
df_votes['voteDate'] = pd.to_datetime(df_votes['voteDate'], 
                                      infer_datetime_format=True)
df_comments['commentDate'] = pd.to_datetime(df_comments['commentDate'], 
                                            infer_datetime_format=True)
df_churn['lastParticipationDate'] = pd.to_datetime(df_churn['lastParticipationDate'], 
                                                   infer_datetime_format=True)

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning:

tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning:

tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



#### Remoção de valores nulos e tratamento de colunas

* As linhas com apenas valores nulos das tabelas **df_interactions** e **df_comments**

In [46]:
df_interactions.dropna(axis=0,
                       how='any',
                       inplace=True)

In [47]:
df_interactions[['liked', 
                 'disliked']] = df_interactions[['liked', 
                                                 'disliked']].astype('bool')

In [48]:
df_interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336959 entries, 0 to 336958
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   employee      336959 non-null  int64 
 1   companyAlias  336959 non-null  object
 2   liked         336959 non-null  bool  
 3   disliked      336959 non-null  bool  
 4   commentId     336959 non-null  object
dtypes: bool(2), int64(1), object(2)
memory usage: 10.9+ MB


In [49]:
df_comments.dropna(axis=0, how='any', thresh=4, inplace=True)

In [50]:
df_comments['carac'] = df_comments['txt'].str.len()

In [51]:
df_comments.drop(columns=['txt'], inplace=True)

In [52]:
df_comments['carac'].fillna(value=0,
                            inplace=True)

In [53]:
df_comments[['likes', 
             'dislikes', 
             'carac']] = df_comments[['likes', 
                                      'dislikes', 
                                      'carac']].astype('int64')

In [54]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82755 entries, 0 to 82754
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   employee      82755 non-null  int64         
 1   companyAlias  82755 non-null  object        
 2   commentId     82755 non-null  object        
 3   likes         82755 non-null  int64         
 4   dislikes      82755 non-null  int64         
 5   commentDate   82755 non-null  datetime64[ns]
 6   carac         82755 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 5.1+ MB


#### Remoção de valores duplicados

In [55]:
df_votes.shape

(221232, 4)

In [56]:
df_votes.drop_duplicates(inplace=False)
df_votes.shape

(221232, 4)

In [57]:
df_comments.shape

(82755, 7)

In [58]:
df_comments.drop_duplicates(inplace=True)
df_comments.shape

(42018, 7)

In [59]:
df_interactions.shape

(336959, 5)

In [60]:
df_interactions.drop_duplicates(inplace=True)
df_interactions.shape

(336952, 5)

In [61]:
df_churn.shape

(4847, 5)

In [62]:
df_churn.drop_duplicates(inplace=True)
df_churn.shape

(4847, 5)

### Construção dos dados

In [63]:
votos = pd.pivot_table(df_votes, 
                       index=['companyAlias', 'employee'],
                       values=['vote'],
                       aggfunc={'vote': [ma.count, np.mean]})

votos.columns = [
                 '.'.join(col).strip() 
                 if col[1] != "" 
                 else col[0] 
                 for col in votos.columns.values
                 ]

In [64]:
fig = px.histogram(votos, 
                   x='vote.mean', 
                   title='Média dos votos por funcionário',
                   nbins=30)
fig.update_layout(bargap=0.2)
fig.show()

In [65]:
fig = px.violin(votos, 
                x='vote.count', 
                title='Quantidade de votos por funcionário',
                box=True, points='all')

fig.show()

In [66]:
fig = px.histogram(df_votes, 
                   x='vote', 
                   title='Distribuição de votos por funcionário')
fig.update_layout(bargap=0.2)
fig.show()

In [67]:
fig = px.violin(df_comments, 
                x='carac', 
                title='Quantidade de caracteres dos comentários',
                points='all')

fig.show()

In [70]:
fig = px.histogram(df_comments, 
                   x='likes', 
                   title='Quantidade de likes dos comentários'
                  )

fig.show()